# GWExPy 新機能総合チュートリアル

`gwexpy` は `gwpy` を拡張し、多チャンネルデータの効率的な処理 (Matrixクラス)、高度な信号処理 (PCA/ICA)、欠損値処理 (Imputation)、および外部ライブラリとの相互運用性 (Interop) を追加したライブラリです。
このノートブックでは、`gwexpy` で追加された主要な機能を一通り紹介します。

In [1]:
import sys
from pathlib import Path

# Allow `import gwexpy` when running this notebook from `examples/`.
cwd = Path.cwd().resolve()
for p in [cwd, *cwd.parents]:
    if (p / "gwexpy").is_dir():
        p_str = str(p)
        if p_str not in sys.path:
            sys.path.insert(0, p_str)
        break

import numpy as np
from astropy import units as u
from gwpy.time import LIGOTimeGPS

# GWExPyの主要クラスをインポート
from gwexpy.timeseries import TimeSeries, TimeSeriesMatrix

# 乱数シードの固定
np.random.seed(42)

## 1. TimeSeriesMatrix: 多チャンネルデータの効率的処理

`TimeSeriesMatrix` は、複数の時系列データ（チャンネル）を1つの行列として扱うためのクラスです。
`gwpy.TimeSeries` のリストや辞書を使うよりも、メモリ効率が良く、一括処理に適しています。

In [2]:
# データの生成: 3つのチャンネル、1000サンプル
n_channels = 3
n_samples = 1000
rate = 100 * u.Hz
dt = 1 / rate

# 形状は (チャンネル数, 1, 時間) = (3, 1, 1000) とします
data = np.random.randn(n_channels, 1, n_samples)

# TimeSeriesMatrix の作成
tsm = TimeSeriesMatrix(
    data,
    dt=dt,
    t0=0,
    unit="V",
    channel_names=["CH1", "CH2", "CH3"]
)

print("TimeSeriesMatrix Summary:")
print(tsm)
print(f"Shape: {tsm.shape}")

TimeSeriesMatrix Summary:
SeriesMatrix(shape=(3, 1, 1000),  name='')
  epoch   : 0.0
  x0      : 0.0 s
  dx      : 0.01 s
  xunit   : s
  samples : 1000

[ Row metadata ]
     name channel unit
key                   
row0                  
row1                  
row2                  

[ Column metadata ]
     name channel unit
key                   
col0                  

[ Elements metadata ]
  unit  name channel  row  col
0    V  None    None    0    0
1    V  None    None    1    0
2    V  None    None    2    0
Shape: (3, 1, 1000)


### スライスとアクセス

行列の一部をスライスすると `TimeSeriesMatrix` が返り、1つの要素を指定すると `TimeSeries` オブジェクトが返されます。

In [3]:
# 最初の2チャンネルを取得
sub_matrix = tsm[:2, :]
print(f"Sub-matrix shape: {sub_matrix.shape}")

# 特定のチャンネル (CH1) を TimeSeries として取得
ts_ch1 = tsm[0, 0]
print("\nExtracted TimeSeries (CH1):")
print(ts_ch1)

Sub-matrix shape: (2, 1, 1000)

Extracted TimeSeries (CH1):
TimeSeries([ 0.49671415, -0.1382643 ,  0.64768854, ...,
             0.64084286, -0.57117899,  0.57258278]
           unit: V,
           t0: 0.0 s,
           dt: 0.01 s,
           name: None,
           channel: None)


## 2. 信号処理: 標準化、白色化、成分分解 (PCA/ICA)

`TimeSeriesMatrix` は、機械学習の前処理によく使われる機能を内蔵しています。

In [4]:
# データの標準化 (Z-score normalization)
tsm_norm = tsm.standardize(axis="time")
print(f"Standardized mean (approx 0): {tsm_norm.value.mean():.2f}")
print(f"Standardized std (approx 1):  {tsm_norm.value.std():.2f}")

# チャンネル間の白色化 (Whitening)
tsm_w, w_model = tsm.whiten_channels()
print("\nWhitened Matrix shape:", tsm_w.shape)

Standardized mean (approx 0): -0.00
Standardized std (approx 1):  1.00

Whitened Matrix shape: (3, 1, 1000)


### 主成分分析 (PCA)
多チャンネルデータの次元削減や特徴抽出に使用できます。

In [5]:
# PCA の実行
scores, pca_model = tsm.pca(return_model=True, n_components=2)

print("PCA Scores shape:", scores.shape)
print("explained_variance_ratio:", pca_model.explained_variance_ratio_)

PCA Scores shape: (2, 1, 1000)
explained_variance_ratio: [0.3517994  0.33009077]


## 3. 欠損値処理 (Imputation)

`gwexpy` は `impute()` メソッドにより、欠損値 (NaN) の補間をサポートしています。

In [6]:
# 欠損値を含むデータを作成
data_nan = data.copy()
data_nan[0, 0, 50:60] = np.nan  # CH1の一部を NaN に

tsm_nan = TimeSeriesMatrix(data_nan, dt=dt, t0=0)

# 線形補間
tsm_imputed = tsm_nan.impute(method="interpolate")

# 確認
original_segment = data[0, 0, 50:60]
imputed_segment = tsm_imputed.value[0, 0, 50:60]

print("Original:", original_segment)
print("Imputed: ", imputed_segment)

Original: [ 0.32408397 -0.38508228 -0.676922    0.61167629  1.03099952
  0.93128012 -0.83921752 -0.30921238  0.33126343  0.97554513]
Imputed:  [-1.64632507 -1.52960999 -1.41289491 -1.29617982 -1.17946474
 -1.06274965 -0.94603457 -0.82931949 -0.7126044  -0.59588932]


## 4. 周波数領域: FrequencySeriesMatrix と Group Delay

`TimeSeriesMatrix` に `fft()` を適用すると `FrequencySeriesMatrix` が得られます。また、`FrequencySeries` には群遅延 (`group_delay`) を計算する機能が追加されています。

In [7]:
# FFT (Matrix全体)
fsm = tsm.fft()
print("FrequencySeriesMatrix:")
print(fsm)

# 特定のチャンネルの FrequencySeries を取得
fs = fsm[0, 0]

# 群遅延 (Group Delay) の計算
# group_delay = -d(phi)/d(omega)
gd = fs.group_delay()
print("\nGroup Delay:")
print(gd)

FrequencySeriesMatrix:
SeriesMatrix(shape=(3, 1, 501),  name='')
  epoch   : 0.0
  x0      : 0.0 Hz
  dx      : 0.1 Hz
  xunit   : Hz
  samples : 501

[ Row metadata ]
     name channel unit
key                   
row0                  
row1                  
row2                  

[ Column metadata ]
     name channel unit
key                   
col0                  

[ Elements metadata ]
  unit  name channel  row  col
0    V  None    None    0    0
1    V  None    None    1    0
2    V  None    None    2    0

Group Delay:
FrequencySeries([-0.3625813 , -1.83569488, -1.54343451, ...,
                  1.86159182,  1.38579454, -0.91715641]
                unit: s,
                f0: 0.0 Hz,
                df: 0.1 Hz,
                epoch: 0.0,
                name: group_delay,
                channel: None)


## 5. 相互運用性 (Interop) の応用例

最後に、これらの処理を行ったデータを PyTorch や Pandas に渡すワークフローの例を示します。

In [8]:
try:
    # 前処理済み (PCAなど) のデータを PyTorch Tensor に変換して学習へ
    import torch
    tensor = tsm.to_torch(dtype=torch.float32)
    print("\nReady for Deep Learning (Torch Tensor):", tensor.shape)
    
    # 単一チャンネルを Pandas で解析
    ts_single = tsm[0, 0]
    df = ts_single.to_pandas()
    print("\nReady for Analysis (Pandas):\n", df.head())
    
except ImportError:
    print("Optional dependencies not installed.")


Ready for Deep Learning (Torch Tensor): torch.Size([3, 1, 1000])

Ready for Analysis (Pandas):
 time_utc
1980-01-06 00:00:19+00:00           0.496714
1980-01-06 00:00:19.010000+00:00   -0.138264
1980-01-06 00:00:19.020000+00:00    0.647689
1980-01-06 00:00:19.030000+00:00    1.523030
1980-01-06 00:00:19.040000+00:00   -0.234153
dtype: float64


## まとめ

`gwexpy` の新機能を使うことで、多チャンネルデータの管理、前処理、そして他の最新ライブラリとの連携が非常にスムーズになります。